#### Notebook to download financial news and headlines from the UIS Refinitiv Eikon subscription.

In [ ]:
import eikon as ek
import csv
import datetime
import time
import pandas as pd

In [ ]:
# MUST HAVE REFINITIV EIKON DESKTOP APP OPEN AND LOGGED IN TO 1 OF UIS ACCOUNTS (user/pw available from library)
ek.set_app_key('6decdfc0fc0a4dbc89f9ab4d81e03532f66a663c')

In [ ]:
# Tickers to download news are read in as RIC codes, but the local OSE ticker will be used in the filename
stocks_df = pd.read_csv('..\..\data\symbology_tickers.csv')
tickers = stocks_df['RIC'].dropna().values.tolist()
tickers[0:10]

In [ ]:
def get_ticker_headlines(ticker, start_date, end_date):
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%dT%H:%M:%S')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%dT%H:%M:%S')
    current_date = start_date
    ticker_OSE = stocks_df[stocks_df["RIC"]==ticker]['LOCAL'].values[0]
    with open('..\\..\\data\\NEWS\\' + ticker_OSE + '.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)

        while current_date > end_date:
            # First get a list of all headlines (contains up to 100)
            print(datetime.datetime.strftime(current_date, '%Y-%m-%dT%H:%M:%S'))
            try:
                hdlns = ek.get_news_headlines(ticker, count=100, date_to=datetime.datetime.strftime(current_date, '%Y-%m-%dT%H:%M:%S'))
            except ek.EikonError as err:
                time.sleep(10)
                continue
            print(hdlns)
            # Next get call the API with each headline ID to get the full story
            for headline in hdlns.itertuples():
                print(headline[2])
                headline = list(headline)
                try:
                    story = ek.get_news_story(headline[3], raw_output=True, debug=False)
                except ek.EikonError as err:
                    time.sleep(10)
                    continue
                headline.insert(0, ticker_OSE)
                headline.append(story)
                writer.writerow(headline)

            # If less than 100 headlines are returned we've reached the end of the period
            if len(hdlns) < 100:
                break
            else:
                current_date = hdlns.index.min().to_pydatetime()

    return


In [ ]:
# loop through all tickers and collect headlines
for ticker in tickers:
    print(ticker)
    get_ticker_headlines(ticker, '2022-04-01T00:00:00', '2021-01-01T00:00:00')